<a href="https://colab.research.google.com/github/abnvanand/ImageInpainting/blob/dev/DenoisingAutoencoder(18).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import mnist
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, InputLayer, Input
import matplotlib.pyplot as plt

In [0]:
# data set of dim 28x28 greyscale images
(x_train, _), (x_test, _) = mnist.load_data()

# original dimensions of train & test datasets
x_train.shape, x_test.shape

In [0]:
# normalize to range [0,1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape to include channel in dimension
# TODO: change channel to 3 when using RGB images
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # by default uses `channels_last` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Dimensions after including channel info
x_train.shape, x_test.shape

In [0]:
def add_noise(data, noise_factor = 0.5):
    # add noise
    res = data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data.shape) 

    # clip to range
    res = np.clip(res, 0., 1.)
    return res

x_train_noisy = add_noise(x_train)
x_test_noisy = add_noise(x_test)

In [0]:
def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm

In [0]:
# KL Divergence regularizer for hidden unit activations
import keras # if using keras
import tensorflow as tf
# from tensorflow import keras # if using tf.keras
kullback_leibler_divergence = keras.losses.kullback_leibler_divergence
K = keras.backend

def kl_divergence_regularizer(activations):
    rho_hat = K.mean(activations, axis=0)
    nprho = np.full((rho_hat.get_shape().as_list()[0],) ,0.05)
    rho = tf.convert_to_tensor(nprho, dtype=tf.float32)
    return 0.01 * (kullback_leibler_divergence(rho, rho_hat)
                 + kullback_leibler_divergence(1 - rho, 1 - rho_hat))

In [0]:
x_train.shape, x_train_noisy.shape, x_test.shape, x_test_noisy.shape

In [0]:
# reshape(flatten) so that we can pass to neural network
x_train = x_train.reshape(len(x_train), np.prod(x_train.shape[1:]))
x_train_noisy = x_train_noisy.reshape(len(x_train_noisy), np.prod(x_train_noisy.shape[1:]))

x_test = x_test.reshape(len(x_test), np.prod(x_test.shape[1:]))
x_test_noisy = x_test_noisy.reshape(len(x_test_noisy), np.prod(x_test_noisy.shape[1:]))

# shape after flattening
x_train.shape, x_train_noisy.shape, x_test.shape, x_test_noisy.shape

In [0]:
# print first 10 test images
n = 10
plt.figure(figsize=(20, 3))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i+1+10)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    

plt.show()

In [0]:
lambdaa = 0.0001
input_layer = Input(shape=(784,))
hidden_layer = Dense(units=392, activation='sigmoid', 
#                     activity_regularizer=kl_divergence_regularizer,
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(input_layer)
output_layer = Dense(units=784, activation='sigmoid',
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(hidden_layer)
da1 = Model(input_layer, output_layer)
da1.summary()

In [0]:
da1.compile('adam', loss='mse')
da1.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True)

In [0]:
predicted = da1.predict(x_test_noisy)

In [0]:
plt.figure(figsize=(40, 4))
for i in range(10):
    # display original images
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display noisy images
    ax = plt.subplot(3, 20, i + 1 + 20)
    plt.imshow(x_test_noisy[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstructed images
    ax = plt.subplot(3, 20, 2*20 +i+ 1)
    plt.imshow(predicted[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)


plt.show()

In [0]:
import pandas as pd

hiddenLayerWeights = pd.DataFrame(da1.layers[1].get_weights()[0]).values
hiddenLayerBias = pd.DataFrame(da1.layers[1].get_weights()[1]).values
hiddenLayerWeights.shape, type(hiddenLayerWeights), hiddenLayerBias.shape, type(hiddenLayerBias)

Wx = np.dot(x_train_noisy, hiddenLayerWeights)
b =  hiddenLayerBias.T
hX = sigmoid(Wx+b)

Wy = np.dot(x_train, hiddenLayerWeights)
hY = sigmoid(Wy + b)

In [0]:
hX.shape, hY.shape

In [0]:
input_layer2 = Input(shape=(392,))
hidden_layer2 = Dense(units=300, activation='sigmoid',
                    activity_regularizer=kl_divergence_regularizer,
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(input_layer2)
output_layer2 = Dense(units=392, activation='sigmoid',
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(hidden_layer2)
da2 = Model(input_layer2, output_layer2)
da2.summary()

In [0]:
da2.compile(optimizer='adam', loss='mse')
da2.fit(hX, hY,
                epochs=50,
                batch_size=128,
                shuffle=True)

In [0]:
# Extract weights
W1, B1 = da1.layers[1].get_weights()
W1_dash, B1_dash = da1.layers[2].get_weights()

W2, B2 = da2.layers[1].get_weights()
W2_dash, B2_dash = da2.layers[2].get_weights()


In [0]:
W1.shape, W2.shape, W2_dash.shape, W1_dash.shape
# B1.shape, B2.shape, B2_dash.shape, B1_dash.shape

In [0]:
W1_dash.shape[1]

In [0]:
lambdaa = 0.0001
input_layer = Input(shape=(W1.shape[0],))
h1 = Dense(units=W1.shape[1], activation='sigmoid', 
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(input_layer)
h2 = Dense(units=W2.shape[1], activation='sigmoid', 
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(h1)
h3 = Dense(units=W2_dash.shape[1], activation='sigmoid', 
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(h2)
output_layer = Dense(units=W1_dash.shape[1], activation='sigmoid',
                    kernel_regularizer=keras.regularizers.l2(lambdaa))(h3)

dnn = Model(input_layer, output_layer)

# Initialize with pretrained weights
# dnn.layers[1].set_weights([W1, B1])
# dnn.layers[2].set_weights([W2, B2])
# dnn.layers[3].set_weights([W2_dash, B2_dash])
# dnn.layers[4].set_weights([W1_dash, B1_dash])

dnn.summary()

In [0]:
# x_sample = x_train[0].reshape(28, 28)
# plt.imshow(x_sample, interpolation='nearest')
# plt.show()

In [0]:
dnn.compile('adam', loss='mse')
dnn.fit(x_train, x_train,
                epochs=50,
                batch_size=50,
                shuffle=True)

In [0]:
predicted2 = dnn.predict(x_train)

In [0]:
plt.figure(figsize=(40, 4))
for i in range(10):
    # display original images
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display noisy images
    ax = plt.subplot(3, 20, i + 1 + 20)
    plt.imshow(x_test_noisy[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstructed images
    ax = plt.subplot(3, 20, 2 * 20 + i + 1)
    plt.imshow(x_train_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  
    
plt.show()

In [0]:
# from keras.datasets import mnist
# import numpy as np

# (x_train, _), (x_test, _) = mnist.load_data()

# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
# x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

# noise_factor = 0.5
# x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
# x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

# x_train_noisy = np.clip(x_train_noisy, 0., 1.)
# x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# n = 10
# plt.figure(figsize=(20, 2))
# for i in range(n):
#     ax = plt.subplot(1, n, i+1)
#     plt.imshow(x_test_noisy[i].reshape(28, 28))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
# plt.show()